# Convolutional Neural Nets

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/olaiya/MLTutorialNotebooks/blob/master/cnn.ipynb)

Let's look at constructing a convolutional neural net. We can use the same MNIST number dataset we used with our mlp and classify the data using CNNs

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from matplotlib import cm

#Want to use version of Tensorflow > 2.0
print('Using Tensorflow version %s' % tf.__version__)

Download MNIST dataset

In [ ]:
#Load the dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Intensity of pixels ranges from 0-255. We need to scale the values
x_train, x_test = x_train / 255.0, x_test / 255.0


In [ ]:
# Reshaping the array to 4-dims so that it can work with the tf.keras.layers.Conv2D API
#The 4 dims are , Num_of Events,image_height, image_width, num_of_channels
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

#Split into training and validation samples
x_train, x_val = x_train[0:55000], x_train[55000:]
y_train, y_val = y_train[0:55000], y_train[55000:]

Build a simple CNN. We will output a convolution layer using 14 different filters (kernels) of size 3x3 with no padding to the images. We will then apply a 3x3 max pooling filter to the convolutional later. The output we will then feed to a mlp with a hidden layer of 128 nodes and then feed that to an output layer of 10 nodes which we will apply the sofmax activation functions

In [ ]:
# Creating a Sequential Model and adding the layers

model = tf.keras.models.Sequential([
    #tf.keras.layers.Conv2D(number_of_filters, kernel_size=(filter size), input_shape=(height, width, num_of_channels))
    tf.keras.layers.Conv2D(14, kernel_size=(3,3), input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPool2D(pool_size=(3, 3)),
    tf.keras.layers.Flatten(), # Flattening the 2D arrays for fully connected layers
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10,activation=tf.nn.softmax)
])

model.summary()

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

num_epochs = 20
batchSize = 1000

history = model.fit(x_train, y_train, 
          validation_data=(x_val, y_val),
          batch_size=batchSize,
          epochs=num_epochs)


Lets look at the loss distribution

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', color='red', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(x_val,  y_val, verbose=2)


Accuracy is high. Can you improve on it?

# CNN Classification using the CIFAR10 Dataset

Let's classify more images by taking the CIFAR10 dataset. The CIFAR10 dataset contains 60,000 colour images in 10 classes, with 6,000 images in each class. The classes are mutually exclusive, so there is no overlap between them. The dataset is divided into 50,000 training images and 10,000 testing images. 

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

# Scale pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0


Let's take a look at some of the data

In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i])
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    plt.xlabel(class_names[train_labels[i][0]])


Our previous images were black and white but now we are working with colour images. This is straight forward, we can just feed the images into our CNN via their colour channels. The images are 32x32 pixels in size and have three colour channels (red,green,blue). So the shape of the images is 32, 32, 3

Let's build a more sophisticated neural net with more convolutional layers. The first convolutional layer will have 32 output channels using a 3x3 filter (kernel) and then reduced in size using maxpooling. The second layer will have 64 output channels, also reduced in size with maxpooling and the final concolutional layer will have a output of 64 channels. We can then feed the output to a mlp with a layer containing 64 neurons and an output of 10 neurons

In [ ]:
model = tf.keras.models.Sequential([
    #tf.keras.layers.Conv2D(number_of_filters, kernel_size=(filter size), input_shape=(height, width, num_of_channels))
    tf.keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(), # Flattening the 2D arrays for fully connected layers
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10)
])

model.summary()

Compile and train the model

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))


Look at the loss

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', color='red', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

### Evaluate the accuracy

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)


Let's demonstrate the power of adding convolutional layers. Each convolutional layer has less information than the previous. Remember it is the final layer you feed into the mlp. So the smaller your final convolutional layer, the smaller and therefore the less parameters you have in your mlp. Of course you require filters (which require parameters) to generate convolutional layers, but the parameters required for the filters is typicall more than compensated by the reduced size of the required mlp. So the beauty of CNNs is that you can reduce the number of parameters required to describe it and not necessarily reduce the performance. You can even improve the performance!

Let's build a more sophisticated neural net with more convolutional layers. The first convolutional layer will have 32 output channels using a 3x3 filter (kernel) and then reduced in size using maxpooling 2x2 filter. The second layer will have 64 output channels using a 3x3 filter, also reduced in size with maxpooling 2x2 filter and the final concolutional layer will have a output of 64 channels using a 3x3 filter. We can then feed the output to a mlp with a layer containing 64 neurons and an output of 10 neurons. Build and run this CNN in the cells below. Use the comments as instructions. Compare the number of parameters of this CNN with the previous CNN and their accuracy profiles.

In [ ]:
model = tf.keras.models.Sequential([
    #tf.keras.layers.Conv2D(number_of_filters, kernel_size=(filter size), input_shape=(height, width, num_of_channels))
    tf.keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(), # Flattening the 2D arrays for fully connected layers
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10)
])

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', color='red', label='Validation loss')
plt.title('Training and validation loss')

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.3, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)


## Exercise: classification of the fashion MNIST dataset

This is another image dataset, but this time of fashion items. Again these are images of 28x28 pixels, black and white with pixel intensities ranging from 0 to 255.

Load the dataset. 

In [ ]:
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

Split the full training set into a validation set and a training set. Also scale the pixel intensities down to the 0-1 range and convert them to floats, by dividing by 255

In [ ]:
X_valid, X_train = X_train_full[:5000] / 255., X_train_full[5000:] / 255.
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.

Need to associate names with the labels

In [ ]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

Lets look at some of the images

In [ ]:
n_rows = 4
n_cols = 10
plt.figure(figsize=(n_cols * 1.2, n_rows * 1.2))
for row in range(n_rows):
    for col in range(n_cols):
        index = n_cols * row + col
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(X_train[index], cmap="binary", interpolation="nearest")
        plt.axis('off')
        plt.title(class_names[y_train[index]], fontsize=12)
plt.subplots_adjust(wspace=0.2, hspace=0.5)


Format data so that it is suitable of a 2D convolutional neural net

In [ ]:
#before
X_train.shape

In [ ]:
X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]

#after
X_train.shape

Construct a 2D neural net to classify the fashion data. Again, use whatever tools you think will help, many convolutional layers, maxpooling, dropout in the mlp..... etc.